In [ ]:
import os.path

import json
from tqdm import tqdm

# Load things

## Define `DATA_DIR`

In [ ]:
try:
    from google.colab import drive

    print(
        "You work on Colab. Gentle as we are, we will mount Drive for you. "
        "It'd help if you allowed this in the popup that opens."
    )
    drive.mount('/content/drive')
    DATA_DIR = os.path.join('drive', 'MyDrive', 'KEDiff', 'data')
except ModuleNotFoundError:
    print("You do not work on Colab")
    DATA_DIR = os.path.join('data')
print(f"{DATA_DIR=}", '-->', os.path.abspath(DATA_DIR))

## Import NER Ensemble _`kners`_

In [ ]:
try:
    from google.colab import drive

    print("You work on Colab, so 'kediff_ner_system.py' needs to be copied over before importing it.")

    import shutil
    shutil.copy(
        src = os.path.join('drive', 'MyDrive', 'KEDiff', 'kediff_ner_system.py'),
        dst = '.'
    )
except ModuleNotFoundError:
    pass

In [ ]:
import kediff_ner_system as kners

In [ ]:
MODELS_DIR = os.path.join(DATA_DIR, 'trained_models')
MODELS_DIR

In [ ]:
classifier_paths = {
    entity_type: os.path.join(MODELS_DIR,
                              kners.CLASSIFIER_MODEL_VERSION,
                              "".join([kners.CLASSIFIER_NAME_BASE, entity_type]),
                              kners.SELECTED_EPOCHS[entity_type])
    for entity_type in kners.ENTITY_TYPES
}
classifier_paths

In [ ]:
{k: os.path.abspath(v) for k, v in classifier_paths.items()}

In [ ]:
ner = kners.KediffNerSystem(classifier_paths, print_debug_messages_to_console=True)

## Import Text and Ground Truth

In [ ]:
with open(os.path.join(DATA_DIR, 'union_dataset.jsonl')) as f:
    lines = [json.loads(line) for line in f]
len(lines)

In [ ]:
line = lines[42]
line

In [ ]:
predicted = ner.ner(line['text'], print_table_to_console=True)

# Apply to entire file

In [ ]:
{
    'id': line['id'],
    'text': line['text'],
    'ground_truth': line['label'],
    'predicted': predicted
}

In [ ]:
# todo remove this later
lines = lines[:1_111]

In [ ]:
for line in tqdm(lines):
    line['predicted'] = ner(line['text'])